In [ ]:
# @title Instalar dependencias

!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI
%cd Retrieval-based-Voice-Conversion-WebUI

!pip install -q -r requirements.txt
!apt install -qq aria2

huggingface_repo = "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {huggingface_repo}hubert_base.pt -o hubert_base.pt

!rm -r /content/sample_data
!mkdir pretrained
!mkdir -p dataset/{raw,split,clean}

from IPython.display import clear_output
clear_output(wait=True)
print("\n---------------------------\n| Dependencias instaladas |\n---------------------------\n")

In [ ]:
# @title Descargar modelos pre-entrenados
MODEL_VERSION = "v0" # @param ["v0", "v2"]
SAMPLE_RATE = "48k" # @param [ "32k", "40k", "48k" ]

for model in ['D', 'G', 'f0D', 'f0G']:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {huggingface_repo}pretrained_{MODEL_VERSION}/{model + SAMPLE_RATE}.pth -d pretrained -o {model + SAMPLE_RATE}.pth

In [ ]:
# @title Conectar con Google Drive (Opcional)

from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# @title Usar la interfaz web
!python infer-web.py --colab

In [ ]:
# @title Entrenar la IA manualmente (Avanzado)

NOMBRE_DEL_MODELO = "xadii"  # @param {type:"string"}
NUMERO_DE_PASOS = 1000 # @param {type:"integer"}
# @markdown Cada cuantos pasos se deberia guardar el modelo en disco en caso de fallos:
GUARDAR_PASOS = 100 # @param {type:"integer"}
SAMPLE_RATE = 48000  # @param [32000, 40000, 48000]
NUMERO_DE_HILOS = 8  # @param {type:"integer"}
ALGORITMO = "harvest"  # @param [ "pm", "dio", "harvest" ]
BATCHSIZE = 32  # @param {type:"slider", min:0, max:40, step:1}
# @markdown Guardar cache del entrenamiento en vram (solo recomendado para entrenamientos con menos de 10 minutos de audio)
CACHE_EN_VRAM = False  # @param {type:"boolean"}
# @markdown Solamente guardar el ultimo ckpt para ahorrar espacio en disco
ULTIMO_CKPT = True  # @param {type:"boolean"}

# aliases de variables de español a ingles
MODELNAME = NOMBRE_DEL_MODELO
MODELEPOCH = NUMERO_DE_PASOS
EPOCHSAVE = GUARDAR_PASOS
THREADCOUNT = NUMERO_DE_HILOS
ALGO = ALGORITMO

# convertir variables del form para ser leibles por los scripts
MODELSAMPLE = str(SAMPLE_RATE // 1000) + "k"
CACHEDATA = int(CACHE_EN_VRAM)
ONLYLATEST = int(ULTIMO_CKPT)

!python3 trainset_preprocess_pipeline_print.py dataset/clean {BITRATE} {THREADCOUNT} logs/{MODELNAME} True
!python3 extract_f0_print.py logs/{MODELNAME} {THREADCOUNT} {ALGO}
!python3 extract_feature_print.py cpu 1 0 0 logs/{MODELNAME}
!python3 train_nsf_sim_cache_sid_load_pretrain.py -e {MODELNAME} -sr {MODELSAMPLE} -f0 1 -bs {BATCHSIZE} -g 0 -te {MODELEPOCH} -se {EPOCHSAVE} -pg pretrained/f0G{MODELSAMPLE}.pth -pd pretrained/f0D{MODELSAMPLE}.pth -l {ONLYLATEST} -c {CACHEDATA}